## Project for Coursera Capstone

## Introduction

In this project we will explore if someone wants to open a business at Manhattan Area of New York City what will be the recommanded neighborhood. 

## Business Problem

The Manhattan area of New York is the economic and administrative center. It is often described as the cultural, financial, media, and entertainment capital of the world. Thus it offers a great opportunity for someone who is trying to open a new shop. However being a diverse metropolitan it also pose certain challanges to figure out where to start a new business.

It would be nice if there is a recommender system which analyze the different neighborhoods and recommend the best area to open the business. This machine learning recommender system will help the small business owner make a informed decision on best areas to open the business.


## Data

The recommender system will use the Foursquare location data to get details of the presense of similar type of business in each neighborhood. Foursquare rest-based API provide details like Venue Name, Catagory, customer likes, etc. which are very useful datapoints to make the decision. 

It will then try to use the clustering algorithm to group the neighborhoods and provide addition details of those neighbourhood. 

The data from Foursquare will be fetched using the longitude and latiatude details of neighborhoods around Manhattan. 